<a href="https://colab.research.google.com/github/AmitNikhade/FSGM-Attacking/blob/main/Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training MNIST classifier

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Normalize pixel values
x_train = x_train / 255.0
x_test = x_test / 255.0
# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
# Create a simple feed-forward neural network model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))


# FGSM attack

In [ ]:

import matplotlib.pyplot as plt
def fgsm_attack(model, x, y_true, epsilon=0.9):
    # Compute gradients of the loss w.r.t. the input
    with tf.GradientTape() as tape:
        tape.watch(x)
        predictions = model(x)
        loss = tf.keras.losses.categorical_crossentropy(y_true, predictions)
    gradients = tape.gradient(loss, x)
    # Get the sign of the gradients
    gradients_sign = tf.sign(gradients)
    # Generate adversarial examples
    x_adv = x + epsilon * gradients_sign
    return x_adv
# Select a random test sample
index = np.random.randint(0, len(x_test))
x_sample = x_test[index].reshape(1, 28, 28)
y_true = y_test[index].reshape(1, 10)  # Reshape y_true to (1, 10)
# Convert x_sample to a TensorFlow tensor
x_sample = tf.convert_to_tensor(x_sample, dtype=tf.float32)
# Generate adversarial example using FGSM attack
x_adv = fgsm_attack(model, x_sample, y_true, epsilon=0.1)
# Predict the label for the original and adversarial example
y_pred_original = np.argmax(model.predict(x_sample), axis=-1)
y_pred_adv = np.argmax(model.predict(x_adv), axis=-1)
# Plot the original and adversarial images
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(x_sample.numpy().reshape(28, 28), cmap='gray')
plt.title(f'Original Image\nPredicted label: {y_pred_original[0]}, True label: {np.argmax(y_true)}')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(x_adv.numpy().reshape(28, 28), cmap='gray')
plt.title(f'Adversarial Image\nPredicted label: {y_pred_adv[0]}, True label: {np.argmax(y_true)}')
plt.axis('off')
plt.tight_layout()
plt.show()